# Coup Game

The idea is to try and solve the card game Coup with both RL and CRM algorithms

In [9]:
import random

In [18]:
# todo: implement game history, pass with each decision players must make
# would be better to just have actions, and no 'event types'.  This means blocking becomes
# new events.

# ok... just rewrite this, maybe using an IDE.
# also, would be a good idea to try a much simpler game with known outcomes, say blackjack or something
# also would be good to generalize to a game tree.
# maybe also a really simple version of the game?

class CoupEvent():
    """ An event in the game. """
    def __init__(self, event_type, player, action, target = None):
        self.event_type = event_type
        self.player = player
        self.action = action
        self.target = target
        
    def __str__(self):

        format_str = "Invalid event: {}-{}-{}-{}"
        
        if self.event_type == "Action":
            if self.target is not None:
                format_str = "Player {1} takes action {2} on {3}."
            else:
                return "Player {1} takes action {2}."
        elif self.event_type == "Block":
            return "Player {1} blocks {3}."
        elif self.event_type == "Call":
            return "Player {1} calls {3}."
        elif self.event_type == "Eliminate":
            return "Player {1} was eliminated.".format(self.player)
        elif self.event_type == "Influence":
            return "Player {1} looses influence.".format(self.player)
        elif self.event_type == "WasCalled":
            return "Player {1} was called by {3} on {2} and "
        elif self.event_type == "WasCalled":
            return "Player {1} was called by {3} on {2} "
        
        return format_str.format(self.event_type, self.player, self.action, self.target)        

class CoupEnv():
    
    """
    
    The game works as follows
    
    2 Cards are delt to each player, with remaining cards in the deck
    
    Players take their turns, where each turn is as follows
    
        Player is presented with "Action" observation
        Player returns (action, target), where target is the target of the action (if needed)
        Players have a chance to block or call 
        Players have a chance to call the block
    
    Observations
    
    (player_id, "Action"): Player should take an action
    
    Actions
    
    (action, target): Player performs action on target
    
    ("Block"): Player blocks player's action
    ("Call"): Player calls player's action or block
    
    Ok, looks like standard response is
    (action, target)
    
    
    """

    # for the moment just consider the 2 player game
    PLAYERS = 2
    
    CARDS = [
        "Duke", 
        "Assassin", 
        "Captain", 
        "Ambassador", 
        "Contessa"]

    # actions are in order of preference.
    ACTIONS = [
        "Assassinate",
        "Coup",
        "Swap",
        "Steal",
        "Taxes",
        "Foreign-Aid",
        "Income"
    ]
    
    CAN_DO = {
        "Duke": "Taxes", 
        "Assassin": "Assassinate",
        "Captain": "Steal",
        "Ambassador": "Swap",
        "Contessa": None
    }
    
    CAN_BLOCK = {
        "Duke": "Foreign-Aid",
        "Assassin": None, 
        "Captain": "Steal",
        "Ambassador": "Steal",
        "Contessa": "Assassinate"
    }
     
    COST = {
        "Income": -1,
        "Foreign-Aid": -2,
        "Coup": 7,
        "Taxes": -3,
        "Assassinate": 3,
        "Steal": 0,
        "Swap": 0
    }

    def __init__(self):
        self.reset()
    
    def reset(self):
        """ Resets environment, dealing new cards, and returning initial observation. """
        cards = self.CARDS[:] * 3
        random.shuffle(cards)
        self.player_coins = [0]*self.PLAYERS
        self.player_cards = [cards[i*2:(i+1)*2] for i in range(self.PLAYERS)]
        self.deck = cards[2*self.PLAYERS:]
        self.game_history = []
        # no initial observation.
        return None
    
    def _get_valid_plays(self, player):
        """ Returns valid plays for player. (assuming truthful)"""
        our_cards = self.player_cards[player]
        our_coins = self.player_coins[player]
        valid_actions = [CAN_DO[card] for card in our_cards if CAN_DO[card] is not None]
        valid_actions = [action for action in valid_actions if self.COST[action] <= our_coins]
        valid_actions.append("Income")
        valid_actions.append("Foreign-Aid") 
        return valid_actions
    
    def _get_valid_blocks(self, player):
        """ Returns valid blocks for player.(assuming truthful)"""
        our_cards = self.player_cards[player]
        valid_blocks = [CAN_BLOCK[card] for card in our_cards if CAN_BLOCK[card] is not None]
        return valid_blocks
    
    def _loose_life(self, player):
        """ player looses a life. """
        if self.player_cards[player] == 1:
            self.player_cards = []
        else:
            # ask player to discard one card.
            self.players[player].act("Discard")
            del self.player_cards[target]
            
        self._log_event(CoupEvent("Discard",player,""))
    
    def _reveal_card(self, player):
        """ player must show a card, then swap it with deck. """
        pass
        
    def _call(self, player_being_called, player_calling, action, is_block=False):
        """ One player calls the other, the player must prove they can perform the said action,
            if they can not they loose a card. """
        
        if is_block:
            valid_blocks = self._get_valid_blocks(player_being_called)
            if action not in valid_blocks:
                self._loose_life(player_being_called)
            else:
                self._reveal_card(player_being_called)   
                self._loose_life(player_calling)
        else:
            valid_actions = self._get_valid_plays(player_being_called)
            if action not in valid_actions:
                self._loose_life(player_being_called)
            else:
                self._reveal_card(player_being_called)
                self._loose_life(player_calling)
     
    
    def _query(self, players, obs):
        """ Querys players in order to see if they want to respond to action"""
        
        (original_player, original_action, target) = obs
        
        for player in players:
            action, _ = self.players[player].act(obs)
            if action is not None:
                if action == "Block":
                    # first we need to check if other player wants to call the block
                    self._query([original_player], (player, 'Block', original_action))
                    pass
                elif action == "Call":
                    self._call(original_player, player, original_action)
                else:
                    print("Invalid response {}.".format(action))

    def _log_event(self, event: CoupEvent):
        """ Records event in game history."""
        self.game_history.append(event)
    
    def play(self, players):
        """ Plays the game, step by step. """
        
        self.players = players
        self.reset()
        current_player = 0
        while True:
            
            # have player make a move
            obs = current_player, "Action", self.player_coins, self.player_cards[current_player]
            action, target = players[current_player].act(obs)
            
            # make sure action is valid
            if action not in self.ACTIONS:
                print("Invalid action {} for player {}".format(action, current_player))
                return
            if self.COST[action] > self.player_coins[current_player]:
                print("Not enough coins for player {} to perform {}".format(current_player, action))
                return
 
            call_order = [(i+current_player) % self.PLAYERS for i in range(self.PLAYERS)]

            # give the player a chance to block the move, when needed give each player a turn to respond 
            # going clockwise from the current player.
            if action == "Income":
                # no blocking possiable
                pass
            elif action == "Foreign-Aid":
                # each player can block
                self._query(call_order, (current_player, action, target))                
            elif action == "Coup":
                # no block, no call
                pass
            elif action == "Taxes":
                # each player can call
                pass
            elif action == "Assassinate":
                # target can block, target can call
                pass
            elif action == "Steal":
                # target can block, target can call
                pass
            elif action == "Swap":
                # anyone can call
                pass
    
    def __str__(self):
        """ Returns description of current game, including hidden state. """
        result = ""
        for i in range(self.PLAYERS):
            result += "Player {}: [{}] {}\n".format(i, self.player_coins[i], self.player_cards[i])
        result += "Deck: {}".format(self.deck)
        return result
    
    def print_event(self, event):
        event_type, action, target = event
        print("Player {}: {}")
    
    def print_history(self):
        for event in self.game_history:
            self.print_action(event)
        
            
def EpsilonTrueAgent():
    # usually takes actions based on cards, but occasionally makes a random action.
    
    def __init__(self, player_id, epsilon=0.1):
        """ @param epsilon Frequency of random calls. """
        self.epsilon = epsilon
        self.player_id = player_id
        
    def act(obs):
        
        player, action, player_coins, our_cards = obs
        our_coins = player_coins[self.player_id]
                
        if player == self.player_id and action == "Action":
            
            # perform a random action
            if random.random() <= self.epsilon:
                
                #another way to do this is to pick random cards, and play as if we have those...
                
                valid_actions = [action for action in ACTIONS if self.COST[action] <= our_coins] 
                # todo: find target
                return ("Action", random.sample(valid_actions), None)
                                                                  
            # check what our cards are and take the best action possible
            valid_actions = self._get_valid_plays(player)
            valid_actions.sort(key = lambda x: ACTIONS.index(x))

            # todo: find target

            print(valid_actions)
            return ("Action", valid_actions[0], None)
                
        blockable_actions = [CAN_BLOCK[card] for card in our_cards if CAN_BLOCK[card] is not None]
            
        if action in blockable_actions or random.random() <= epsilon:
            return ("Block", action, None)
            
        return ("Pass", None, None)
    
    

def ImDukeAgent():
    # always call duke, and use coup on 'strongest' player
    
    def __init__(self, player_id):
        self.player_id = player_id
    
    def act(obs):
        
        player, action = obs
                
        if player == self.player_id and action == "Action":
            return "Taxes"
        if action == "Foreign-Aid":
            return "Block"
            
        return "Pass"
    
    

In [20]:
env = CoupEnv()
print(env)

Player 0: [0] ['Assassin', 'Contessa']
Player 1: [0] ['Captain', 'Captain']
Deck: ['Duke', 'Ambassador', 'Duke', 'Assassin', 'Ambassador', 'Duke', 'Ambassador', 'Assassin', 'Captain', 'Contessa', 'Contessa']


In [ ]:
# simplifcations:
# game has only 10 rounds.
# there are 2 players.
# number of decisions for each round is

# P1 Act (7) P2 Call / reverse call (5) - i.e. call-reverse[yes/no], call-no_reverse[yes/no], no-call 
# then the same for P2

# this gives 35*35=1225 states per round and ~2e12 states for 4 rounds, which doesn't work. (this is an 8 terrabyte table)
# also 4 rounds isn't enough... so some kind of state abstraction is required.
# 3 round is ~2 billion (2e9), which would fit into memory, but only just... however we'd never get enough iterations to fill the table.
# maybe a better solution is to use function approximation for the regret function, this could be interesting...
# otherwise I'd need to simplify the game further, and I'm not sure how to do that? Seems like most of the calls matter...


# yeah, probably just use this: https://arxiv.org/pdf/1811.00164.pdf

# hmm, ok I get this now, will need a much simpler game to test on.  Something that has hidden information.

# these guys do the 10e18 poker http://papers.nips.cc/paper/3306-regret-minimization-in-games-with-incomplete-information.pdf
# there is a simpler poker though

# kuhn poker is the way to go, has known nash.
#https://en.wikipedia.org/wiki/Kuhn_poker

def RLOptimizer():
    # Try to find a good policy via RL methods.
    # would be good to try a simple state based one, and one with LSTM units.
    # maybe also train them against standard agents.
    
class CRMOptimizer():
    # Calculate regrets, and minimize using CRM algorithm.
    
    # Looks like our game state is complete history? hmm... that's problematic. Maybe we need to reduce the rounds?
    
    # so the idea is to record regret for each game state, then act to minimize this.
    